In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
import paramiko
import datetime
from datetime import date
import re
from collections import OrderedDict
import simpler_sf

simpler_sf.simple_salesforce()
import simple_salesforce
from simple_salesforce.exceptions import SalesforceMalformedRequest

c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [4]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [5]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy) == 1:
    mes_hoy = "0" + mes_hoy
if len(dia_hoy) == 1:
    dia_hoy = "0" + dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost = "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
# nombreArchivoHost = "VentaCARE_ASSISTANCE_" + '22112023' + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost

# Direccion descarga local
direccionDescargaLocal = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost
)

print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)

2024-05-27
27052024
VentaCARE_ASSISTANCE_27052024.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_27052024.xlsx


In [6]:
host = "sftp.bciseguros.cl"
puerto = 22
transporte = paramiko.Transport((host, puerto))
usuario = "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None, usuario, clave)
sftp = paramiko.SFTPClient.from_transport(transporte)

Coneccion a carpeta sft y rescate de archivo


In [7]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath, localpath)

In [8]:
if sftp:
    sftp.close()
if transporte:
    transporte.close()

Inicio Manipulacion de Excel descargado


In [9]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape

c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(25170, 17)

In [10]:
archivoBCI.dtypes

POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [11]:
archivoBCI.RUT_ASEGURADO.fillna(archivoBCI["RUT_TITULAR"], inplace=True)
archivoBCI.DV_ASEGURADO.fillna(archivoBCI["DV_TITULAR"], inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)
archivoBCI.DIREC.fillna("", inplace=True)
archivoBCI.NOMBRE_ASEGURADO.fillna("NN NN", inplace=True)
archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2813446,2024-05-06,2025-05-05,SEMO,M1,SEGURO EXÁMENES TOTAL,20434558,9,20434558,9,FEMENINO,TITULAR MASCULINO,FRANCISCA CARRASCO RIVERA,2000-05-29,Carrascofrancisca2905@gmail.com,SANTIAGO,972054108
1,2813443,2024-05-06,2025-05-05,SEMO,M2,SEGURO MEDICAMENTOS FULL,20434558,9,20434558,9,FEMENINO,TITULAR MASCULINO,FRANCISCA CARRASCO RIVERA,2000-05-29,Carrascofrancisca2905@gmail.com,SANTIAGO,972054108
2,2809407,2024-04-25,2025-04-24,SEMO,M1,SEGURO EXÁMENES TOTAL,20486750,K,20486750,K,FEMENINO,TITULAR MASCULINO,Romina Andrea Rodriguez Yaez,2000-04-15,amberlyn.henriquez@gmail.com,.,976364243
3,2815760,2024-05-10,2025-05-09,SEMO,M2,SEGURO MEDICAMENTOS FULL,20656908,5,20656908,5,MASCULINO,TITULAR MASCULINO,SEBASTIAN IGNACIO LEON MUNOZ,2001-01-09,SEBASTIANLEON.MF@GMAIL.COM,PARVA 798,996743569
4,2810989,2024-04-29,2025-04-28,SEMO,M2,SEGURO MEDICAMENTOS FULL,21594507,3,21594507,3,FEMENINO,TITULAR MASCULINO,SCARLETH RACHEL KETTERER VALENZUELA,2004-06-10,valenzuelascarleth91@gmail.com,SANTIAGO,978052638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25115,2629905,2023-08-06,2024-08-05,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,8077683,7,8208790,7,FEMENINO,ESPOSA,UBERLINDA ESTER MERY ESPEJO,1961-10-06,,SANTIAGO,0
25116,2646281,2023-09-06,2024-09-05,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,15416752,8,25526707,8,MASCULINO,HIJO,BENJAMN JOSUE JENERAL PREZ,2016-10-04,CECY_PEREZ767@HOTMAIL.COM,NO INFORMADO,0
25117,2646281,2023-09-06,2024-09-05,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,15416752,8,25526707,8,MASCULINO,HIJO,BENJAMN JOSUE JENERAL PREZ,2016-10-04,CECY_PEREZ767@HOTMAIL.COM,NO INFORMADO,0
25118,2646281,2023-09-06,2024-09-05,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,15416752,8,25526707,8,MASCULINO,HIJO,BENJAMN JOSUE JENERAL PREZ,2016-10-04,CECY_PEREZ767@HOTMAIL.COM,NO INFORMADO,0


In [12]:
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def nombreFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[0] + " " + nombreCompleto[1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[0]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


def apellidoFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[-2] + " " + nombreCompleto[-1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[-1]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


""" def apellidoFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)==1:
    return apellidos[0]
  elif len(apellidos)==2:
    nombresJuntos = apellidos[0] + " " + apellidos[1]
    return nombresJuntos
    
  elif len(apellidos)==3:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2]
  else:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] """

# archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
# archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3]
# print (apellidos)

' def apellidoFuncion(nombreCompleto):\n  nombreCompleto = str(nombreCompleto)\n  apellidos = re.split(\'\\W+\', nombreCompleto)\n  if len(apellidos)==1:\n    return apellidos[0]\n  elif len(apellidos)==2:\n    nombresJuntos = apellidos[0] + " " + apellidos[1]\n    return nombresJuntos\n    \n  elif len(apellidos)==3:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2]\n  else:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] '

In [13]:
archivoBCI["Rut Titular"] = archivoBCI["RUT_TITULAR"].astype(str) + archivoBCI[
    "DV_TITULAR"
].astype(str)
#archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "").str.upper()
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace("-", "")
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(":", "")
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace("_", "")
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace("–", "")
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(".", "")
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "")
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.upper()
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.strip()

archivoBCI["Rut Carga"] = archivoBCI["RUT_ASEGURADO"].astype(str) + archivoBCI[
    "DV_ASEGURADO"
].astype(str)
#archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("-", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(":", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("_", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("–", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(".", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.strip()

archivoBCI["poliza"] = archivoBCI["POLIZA"].astype(str)

general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)
archivoBCI["sexo"] = archivoBCI["SEXO"].astype(str).str.title()

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI["email"] = archivoBCI["email"].apply(lambda y: isValid(y))
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion = {
    "no informado": "",
    "SN": "",
    "sn": "",
    "s/d": "",
    "1 1": "",
    "0 0": "",
    "sn 0": "",
}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

general_Celular = {
    0: "",
}
archivoBCI.replace({"CELULAR": general_Celular}, inplace=True)
archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str).str.lower()
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace(" ", "")


archivoBCI["nombres"] = archivoBCI["NOMBRE_ASEGURADO"].str.split(n=2)
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].str.rsplit(n=2)
archivoBCI["nombres"] = (
    archivoBCI["nombres"].apply(lambda y: nombreFuncion(y)).str.title()
)
archivoBCI["apellidos"] = (
    archivoBCI["apellidos"].apply(lambda y: apellidoFuncion(y)).str.title()
)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_17420\3448275711.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_17420\3448275711.py:27: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_17420\3448275711.py:91: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = ar

In [14]:
archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,CELULAR,Rut Titular,Rut Carga,poliza,sexo,email,direccion,celular,nombres,apellidos
0,2813446,2024-05-06,2025-05-05,SEMO,M1,SEGURO EXÁMENES TOTAL,20434558,9,20434558,9,...,972054108,204345589,204345589,2813446,Mujer,carrascofrancisca2905@gmail.com,Santiago,972054108,Francisca Carrasco,Carrasco Rivera
1,2813443,2024-05-06,2025-05-05,SEMO,M2,SEGURO MEDICAMENTOS FULL,20434558,9,20434558,9,...,972054108,204345589,204345589,2813443,Mujer,carrascofrancisca2905@gmail.com,Santiago,972054108,Francisca Carrasco,Carrasco Rivera
2,2809407,2024-04-25,2025-04-24,SEMO,M1,SEGURO EXÁMENES TOTAL,20486750,K,20486750,K,...,976364243,20486750K,20486750K,2809407,Mujer,amberlyn.henriquez@gmail.com,.,976364243,Romina Andrea,Rodriguez Yaez
3,2815760,2024-05-10,2025-05-09,SEMO,M2,SEGURO MEDICAMENTOS FULL,20656908,5,20656908,5,...,996743569,206569085,206569085,2815760,Hombre,sebastianleon.mf@gmail.com,Parva 798,996743569,Sebastian Ignacio,Leon Munoz
4,2810989,2024-04-29,2025-04-28,SEMO,M2,SEGURO MEDICAMENTOS FULL,21594507,3,21594507,3,...,978052638,215945073,215945073,2810989,Mujer,valenzuelascarleth91@gmail.com,Santiago,978052638,Scarleth Rachel,Ketterer Valenzuela
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25115,2629905,2023-08-06,2024-08-05,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,8077683,7,8208790,7,...,,80776837,82087907,2629905,Mujer,,Santiago,,Uberlinda Ester,Mery Espejo
25116,2646281,2023-09-06,2024-09-05,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,15416752,8,25526707,8,...,,154167528,255267078,2646281,Hombre,cecy_perez767@hotmail.com,,,Benjamn Josue,Jeneral Prez
25117,2646281,2023-09-06,2024-09-05,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,15416752,8,25526707,8,...,,154167528,255267078,2646281,Hombre,cecy_perez767@hotmail.com,,,Benjamn Josue,Jeneral Prez
25118,2646281,2023-09-06,2024-09-05,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,15416752,8,25526707,8,...,,154167528,255267078,2646281,Hombre,cecy_perez767@hotmail.com,,,Benjamn Josue,Jeneral Prez


In [15]:
# archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
# archivoBCI.to_excel("F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx",index=False)

In [16]:
def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value


""" def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value """

' def revisaDict(ver):\n    numbers = ver\n    for key, value in numbers.items():\n        print(key, "->", value)\n        if key == "IdentificationId__pc":\n            return value '

In [17]:
""" results = sf.query_all(
    
    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Seguros Individuales'
  
)
pacienteProgr = pd.DataFrame(results["records"])
# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc
# String acctName = carol.Account.Name;

pacienteProgr["sfRut"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))

pacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()

pacienteProgr = pacienteProgr[["sfRut",]]


pacienteProgr.shape """

' results = sf.query_all(\n    \n    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = \'BCI Seguros Individuales\'\n  \n)\npacienteProgr = pd.DataFrame(results["records"])\n# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc\n# String acctName = carol.Account.Name;\n\npacienteProgr["sfRut"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))\n\npacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()\n\npacienteProgr = pacienteProgr[["sfRut",]]\n\n\npacienteProgr.shape '

In [18]:
# pacienteProgr.head()

In [20]:
campaña_BCI = "BCI Seguros Individuales"
query_pacientePrograma = (
    "select Id, Campa_a__c, Paciente__r.IdentificationId__pc, N_Documento_Titular__c,Paciente__r.FirstName, Paciente__r.LastName,Empresa__r.P_liza__c, Empresa__r.Rut_Empresa__c, Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña_BCI
    + "'"
)
PAcientes_q_ya_estan0 = sf.smart_query(query_pacientePrograma)

campaña_BCI = "BCI Seguros Colectivos"
query_pacientePrograma1 = (
    "select Id, Campa_a__c, Paciente__r.IdentificationId__pc, N_Documento_Titular__c,Paciente__r.FirstName, Paciente__r.LastName,Empresa__r.P_liza__c, Empresa__r.Rut_Empresa__c, Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña_BCI
    + "'"
)
PAcientes_q_ya_estan1 = sf.smart_query(query_pacientePrograma1)

campaña_BCI = "BCI Seguros Colaboradores"
query_pacientePrograma2 = (
    "select Id, Campa_a__c, Paciente__r.IdentificationId__pc, N_Documento_Titular__c,Paciente__r.FirstName, Paciente__r.LastName,Empresa__r.P_liza__c, Empresa__r.Rut_Empresa__c, Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña_BCI
    + "'"
)
PAcientes_q_ya_estan2 = sf.smart_query(query_pacientePrograma2)


PAcientes_q_ya_estan = pd.concat(
    [
        PAcientes_q_ya_estan0,
        PAcientes_q_ya_estan1,
        PAcientes_q_ya_estan2,
    ],
    axis=0,
)

PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["Paciente__r.IdentificationId__pc"]
PAcientes_q_ya_estan["Rut"] = PAcientes_q_ya_estan["Paciente__r.IdentificationId__pc"]
PAcientes_q_ya_estan.head()

PAcientes_q_ya_estan.to_excel("F:/Empresas SF/reportes/rutsBci.xlsx", index=False)


PAcientes_q_ya_estan.shape

100%|██████████| 1/1 [00:03<00:00,  3.14s/it]


(40865, 13)

In [21]:
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].astype(str).str.upper()
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].str.replace(" ", "")
PAcientes_q_ya_estan["sfRut"] = PAcientes_q_ya_estan["sfRut"].str.replace("-", "")

archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str).str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("-", "")

archivoBCI["RutEstaoNo"] = archivoBCI["Rut Carga"].isin(PAcientes_q_ya_estan["sfRut"])

archivoBCI = archivoBCI[archivoBCI["RutEstaoNo"] == False]


""" archivoBCI.drop(
    archivoBCI[
        (archivoBCI["nombres"] == "Nn Nn") & (archivoBCI["apellidos"] == "Nn Nn")
    ].index,
    inplace=True,
) """
archivoBCI = archivoBCI.drop_duplicates(["Rut Carga"], keep="last")

archivoBCI = archivoBCI[
    [
        "Rut Carga",
        "sexo",
        "direccion",
        "email",
        "celular",
        "nombres",
        "apellidos",
        "Rut Titular",
    ]
]

archivoBCI = archivoBCI.rename(
    columns={
        "Rut Carga": "IdentificationId__pc",
        "sexo": "HealthCloudGA__Gender__pc",
        "direccion": "BillingStreet",
        "email": "PersonEmail",
        "celular": "Phone",
        "nombres": "FirstName",
        "apellidos": "LastName",
    }
)
archivoBCI["BillingCountry"] = "Chile"
archivoBCI["Identification_Type__c"] = "RUT"
archivoBCI["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
archivoBCI["RecordTypeId"] = "0128c000002Byp4AAC"
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI["Validaci_n__c"] = "Validada"
archivoBCI["Email_Seteado__c"] = np.where(
    archivoBCI["PersonEmail"] != "", "True", "False"
)
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI.to_excel(
    "F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx", index=False
)


archivoBCI.head()

,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,Rut Titular,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c
5,266894023,Mujer,.,mabely3004@gmail.com,976841142,Bexy Diaz,Bexy Diaz,266894023,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
10,223698689,Mujer,.,,,Matas Nicols,Soto Bascur,13031807K,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False
12,208248650,Mujer,.,juanfusion@gmail.com,,Javiera Andrea,Soto Bascur,208248650,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
14,215639797,Mujer,.,jaimecastillol@hotmail.com,,Florencia Castillo,Florencia Castillo,215639797,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
105,1007,Hombre,Pasaje Manco Capac Nro 1745,paulitalorena@hotmail.com,,Rut De,Prueba Certificacion,1007,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True


In [22]:
archivoBCI.head()


,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,Rut Titular,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c
5,266894023,Mujer,.,mabely3004@gmail.com,976841142,Bexy Diaz,Bexy Diaz,266894023,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
10,223698689,Mujer,.,,,Matas Nicols,Soto Bascur,13031807K,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False
12,208248650,Mujer,.,juanfusion@gmail.com,,Javiera Andrea,Soto Bascur,208248650,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
14,215639797,Mujer,.,jaimecastillol@hotmail.com,,Florencia Castillo,Florencia Castillo,215639797,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True
105,1007,Hombre,Pasaje Manco Capac Nro 1745,paulitalorena@hotmail.com,,Rut De,Prueba Certificacion,1007,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True


In [23]:
""" archivoDeCarga = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + str(archivoDia) + ".xlsx"
)

archivoBCI.to_excel(archivoDeCarga, index=False) """

' archivoDeCarga = (\n    "F:/Procesos Automaticos/BCI Canal Individual/" + str(archivoDia) + ".xlsx"\n)\n\narchivoBCI.to_excel(archivoDeCarga, index=False) '

In [24]:
""" derty = sf.quick_search("153298939")
#print(derty)

revisaDict(derty)
 """

""" soql_query =  select Id, Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Seguros Individuales' 
soql_result = sf.query_all(soql_query)
df = pd.DataFrame(soql_result.get("records"))
df = df.drop(columns=["attributes"])
#df = df.drop(columns="Id")
list_of_records = df.to_dict("index") """

""" derty = sf.quick_search("153298939")
print(derty) """

""" for listado in list_of_records:
    list_of_records[listado]["Paciente__r"]["IdentificationId__pc"]
    juanka =  len(list_of_records)
    #print(list_of_records[listado]["Paciente__r"]["IdentificationId__pc"])
    print(list_of_records[listado]["Id"]) 
    
list_of_records[1]"""

' for listado in list_of_records:\n    list_of_records[listado]["Paciente__r"]["IdentificationId__pc"]\n    juanka =  len(list_of_records)\n    #print(list_of_records[listado]["Paciente__r"]["IdentificationId__pc"])\n    print(list_of_records[listado]["Id"]) \n    \nlist_of_records[1]'

Weeeeeeeeeeeenooo


In [25]:
# rut = "124942012"
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = (
            "FIND {"
            + rut
            + "} RETURNING Account(Id,IdentificationId__pc WHERE IdentificationId__pc = '"
            + rut
            + "')"
        )
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
                return "NoNe"
        else:
            print("vacio")
            return "NoNe"
            
    else:
        return "NoNe"


def cambiaranone(Id2):
    if Id2 is not None and len(Id2) >= 7:
        return Id2
    else:
        return "NoNe"

In [26]:
archivoBCI.shape

(158, 15)

In [27]:
archivoBCI["Paciente__c"] = archivoBCI["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)


archivoBCI["Empresa__c"] = "a3yRN0000006rMRYAY"
archivoBCI["N_Documento_Titular__c"] = archivoBCI["Rut Titular"]

archivoBCI["esta"] = archivoBCI["Paciente__c"].apply(lambda y: cambiaranone(y))
#archivoBCI = archivoBCI[archivoBCI["esta"] == "NoNe"]


# Empresa__c, Paciente__c, N_Documento_Titular__c,
archivoBCI.head()

vacio
001RN00000A6WM4YAN 223698689
001RN00000A6ZqCYAV 208248650
vacio
001RN00000A6fE2YAJ 13031807K
001RN00000A6ffRYAR 136559362
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002bJsGnAAK 8538712K
vacio
0018c00002axmJuAAI 90436821
vacio
vacio
vacio
001RN00000A6eZiYAJ 95374573
vacio
001RN00000A6X20YAF 98152962
vacio
vacio
vacio
001RN00000A6avuYAB 106573735
001RN00000A6gLNYAZ 107190589
001RN00000A6el0YAB 137634589
001RN00000A6fp7YAB 134278277
vacio
vacio
0018c00002eU2mZAAS 135088188
001RN00000A6VoAYAV 125445470
0018c00002bKJjQAAW 132666792
001RN00000A6gGXYAZ 136337173
vacio
0018c00002eTxelAAC 126998392
001RN00000A6U47YAF 119850428
vacio
001RN00000A6VEhYAN 112540504
0018c00002aza4PAAQ 118602250
vacio
0018c00002aza5qAAA 115179004
vacio
vacio
001RN00000A6c86YAB 165301390
vacio
001RN00000A6WtyYAF 157619411
001RN00000A6W97YAF 157831615
001RN00000A6cckYAB 155461535
001RN00000A6cePYAR 161684015
vacio
001RN00000A6g6rYAB 156358568
001RN00000A6aZMYAZ 164802817
vacio
vacio
vacio
001RN00000A6cT4YAJ

,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,Rut Titular,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Empresa__c,N_Documento_Titular__c,esta
5,266894023,Mujer,.,mabely3004@gmail.com,976841142,Bexy Diaz,Bexy Diaz,266894023,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,NoNe,a3yRN0000006rMRYAY,266894023,NoNe
10,223698689,Mujer,.,,,Matas Nicols,Soto Bascur,13031807K,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,001RN00000A6WM4YAN,a3yRN0000006rMRYAY,13031807K,001RN00000A6WM4YAN
12,208248650,Mujer,.,juanfusion@gmail.com,,Javiera Andrea,Soto Bascur,208248650,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN00000A6ZqCYAV,a3yRN0000006rMRYAY,208248650,001RN00000A6ZqCYAV
14,215639797,Mujer,.,jaimecastillol@hotmail.com,,Florencia Castillo,Florencia Castillo,215639797,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,NoNe,a3yRN0000006rMRYAY,215639797,NoNe
105,1007,Hombre,Pasaje Manco Capac Nro 1745,paulitalorena@hotmail.com,,Rut De,Prueba Certificacion,1007,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,NoNe,a3yRN0000006rMRYAY,1007,NoNe


In [28]:
""" archivoBCI = archivoBCI[archivoBCI["esta"] != "NoNe"]
archivoBCI.head() """

' archivoBCI = archivoBCI[archivoBCI["esta"] != "NoNe"]\narchivoBCI.head() '

In [29]:
archivoDeCarga = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + str(archivoDia) + ".xlsx"
)

archivoBCI.to_excel(archivoDeCarga, index=False)

archivoBCI.head()

,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,Rut Titular,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Empresa__c,N_Documento_Titular__c,esta
5,266894023,Mujer,.,mabely3004@gmail.com,976841142,Bexy Diaz,Bexy Diaz,266894023,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,NoNe,a3yRN0000006rMRYAY,266894023,NoNe
10,223698689,Mujer,.,,,Matas Nicols,Soto Bascur,13031807K,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,001RN00000A6WM4YAN,a3yRN0000006rMRYAY,13031807K,001RN00000A6WM4YAN
12,208248650,Mujer,.,juanfusion@gmail.com,,Javiera Andrea,Soto Bascur,208248650,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,001RN00000A6ZqCYAV,a3yRN0000006rMRYAY,208248650,001RN00000A6ZqCYAV
14,215639797,Mujer,.,jaimecastillol@hotmail.com,,Florencia Castillo,Florencia Castillo,215639797,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,NoNe,a3yRN0000006rMRYAY,215639797,NoNe
105,1007,Hombre,Pasaje Manco Capac Nro 1745,paulitalorena@hotmail.com,,Rut De,Prueba Certificacion,1007,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,NoNe,a3yRN0000006rMRYAY,1007,NoNe


In [30]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")

    derty = None
    # vListado[9] = datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S")

    if vListado[9] == "":
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9],
                        # "PersonBirthdate":datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S").formatGMT('yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'' + '"'),
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    # Verify that the Contact was created successfully
    # created_contact = sf.Contact.get(sf_res['id'])
    # print(created_contact)


""" sf_res =None
try:
    # Create the new Contact record in Salesforce
    sf_res = sf.Contact.create(new_contact)
except SalesforceMalformedRequest as e:
    if 'duplicateResut' in e[0]:
        print(f"Malformed request error: {e}")
        # Handle the error here as needed
        #Bypass SF Email duplicate rule by removing the email to create a SF Lead
        del new_contact['Email']
        sf_res = sf.Contact.create(new_contact)


if sf_res is not None:
    # Verify that the Contact was created successfully
    created_contact = sf.Contact.get(sf_res['id'])
    print(created_contact) """

' sf_res =None\ntry:\n    # Create the new Contact record in Salesforce\n    sf_res = sf.Contact.create(new_contact)\nexcept SalesforceMalformedRequest as e:\n    if \'duplicateResut\' in e[0]:\n        print(f"Malformed request error: {e}")\n        # Handle the error here as needed\n        #Bypass SF Email duplicate rule by removing the email to create a SF Lead\n        del new_contact[\'Email\']\n        sf_res = sf.Contact.create(new_contact)\n\n\nif sf_res is not None:\n    # Verify that the Contact was created successfully\n    created_contact = sf.Contact.get(sf_res[\'id\'])\n    print(created_contact) '

In [31]:
Subir_pacientes = archivoBCI
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonBirthdate"].apply(lambda x: x.isoformat() + "Z")
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)
""" Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
) """

Malformed request error: Malformed request https://careassistance.my.salesforce.com/services/data/v42.0/sobjects/Account/. Response content: [{'message': 'se encontró un valor duplicado: IdentificationId__c duplica el valor en el registro con Id.: 003RN00000QeRliYAF', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
001RN00000A6WM4YAN 223698689
Malformed request error: Malformed request https://careassistance.my.salesforce.com/services/data/v42.0/sobjects/Account/. Response content: [{'message': 'se encontró un valor duplicado: IdentificationId__c duplica el valor en el registro con Id.: 003RN00000QeS1pYAF', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
001RN00000A6ZqCYAV 208248650
Malformed request error: Malformed request https://careassistance.my.salesforce.com/services/data/v42.0/sobjects/Account/. Response content: [{'message': 'se encontró un valor duplicado: IdentificationId__c duplica el valor en el registro con Id.: 0038c00003JnKlCAAV', 'errorCode': 'DUPLICATE_VALUE', 'fields':

' Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(\n    lambda x: agregarPaciente(x)\n) '

In [32]:
PAcientes_q_ya_estan.head()
# PAcientes_q_ya_estan.shape

,Id,Campa_a__c,Paciente__r.IdentificationId__pc,Paciente__r.FirstName,Paciente__r.LastName,N_Documento_Titular__c,Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c,Empresa__r.Identificador__c,Convenio_Activo__c,Relaci_n_Con_Asegurado__c,sfRut,Rut
0,a40RN000001i6VwYAI,BCI Seguros Individuales,230355703,Laura Larran,Larran Barros,104079717,-,-,BCI Seguros,True,Carga,230355703,230355703
1,a40RN000001i6VxYAI,BCI Seguros Individuales,253218924,Clara Larran,Larran Barros,104079717,-,-,BCI Seguros,True,Carga,253218924,253218924
2,a40RN000001i6VyYAI,BCI Seguros Individuales,130282172,"Barros Latorre,",Maria Macarena,104079717,-,-,BCI Seguros,True,Carga,130282172,130282172
3,a40RN000001i6VzYAI,BCI Seguros Individuales,220583597,Mara Gracia,Larran Barros,104079717,-,-,BCI Seguros,True,Carga,220583597,220583597
4,a40RN000001i6W0YAI,BCI Seguros Individuales,215818624,Juliana Sanhueza,Sanhueza Magarinos,215818624,-,-,BCI Seguros,True,Titular,215818624,215818624


In [33]:
rut_yidEmCoCo = pd.DataFrame()
rut_yidEmCoCo["Id empreconcon"] = PAcientes_q_ya_estan["Id"]
rut_yidEmCoCo["rut"] = PAcientes_q_ya_estan["Paciente__r.IdentificationId__pc"]

rut_yidEmCoCo.shape

(40865, 2)

In [34]:
rut_yidEmCoCo = rut_yidEmCoCo.drop_duplicates(["rut"], keep="last")
rut_yidEmCoCo.head()

,Id empreconcon,rut
0,a40RN000001i6VwYAI,230355703
1,a40RN000001i6VxYAI,253218924
2,a40RN000001i6VyYAI,130282172
3,a40RN000001i6VzYAI,220583597
4,a40RN000001i6W0YAI,215818624


In [35]:
rut_yidEmCoCo.shape

(40716, 2)

In [36]:
PAcientes_q_ya_estan["RutEstaoNo"] = PAcientes_q_ya_estan["Id"].isin(
    rut_yidEmCoCo["Id empreconcon"]
)
PAcientes_q_ya_estan.shape

(40865, 14)

In [37]:
noestan = PAcientes_q_ya_estan[
    (PAcientes_q_ya_estan["RutEstaoNo"] == False)
    & (PAcientes_q_ya_estan["Relaci_n_Con_Asegurado__c"] != "Titular")
]
rutsBCI = PAcientes_q_ya_estan[(PAcientes_q_ya_estan["RutEstaoNo"] == True)]
noestan.shape

(89, 14)

In [38]:
archivoDeCarga = "F:/Procesos Automaticos/BCI Canal Individual/repetidos.xlsx"

noestan.to_excel(archivoDeCarga, index=False)

In [39]:
archivoDeCarga2 = "F:/Empresas SF/reportes/rutsBci.xlsx"
rutsBCI.to_excel(archivoDeCarga2, index=False)

In [40]:
archivoDeCarga3 = "F:/Empresas SF/reportes/rutsBci2.xlsx"
PAcientes_q_ya_estan.to_excel(archivoDeCarga3, index=False)

In [41]:
""" sf.Account.create(
    {
        "IdentificationId__pc": "12345678910",
        "HealthCloudGA__Gender__pc": "Mujer",
        "FirstName": "johan",
        "LastName": "Smith",
        "PersonEmail": "example@example.com",
        "BillingCountry": "Chile",
        "Identification_Type__c": "RUT",
        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",
        "RecordTypeId": "0128c000002Byp4AAC",
    }
)
 """

# sf.Account.update(1, {'Name': 'Test Account', ContactId: contact_id}) # doctest: +SKIP

' sf.Account.create(\n    {\n        "IdentificationId__pc": "12345678910",\n        "HealthCloudGA__Gender__pc": "Mujer",\n        "FirstName": "johan",\n        "LastName": "Smith",\n        "PersonEmail": "example@example.com",\n        "BillingCountry": "Chile",\n        "Identification_Type__c": "RUT",\n        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",\n        "RecordTypeId": "0128c000002Byp4AAC",\n    }\n)\n '

Codigo que dio github Copilot para la solucion del error de 0 en envio de correos


In [42]:
""" import win32com.client as win32
import os
import shutil

nombre_reporte = "Vida Security - Ferronor"
nombre_carpeta = "F:/Reportes SF/Vida Security/"
nombre_archivo_final = nombre_carpeta + str(dia) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dia)
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

# create Excel application object
excel = win32.gencache.EnsureDispatch('Excel.Application')

# set ScreenUpdating property to False to hide Excel application
excel.ScreenUpdating = False

# open workbook
workbook = excel.Workbooks.Open(nombre_archivo_final)

# do some work with the workbook

# save and close workbook
workbook.Save()
workbook.Close()

# set ScreenUpdating property to True to show Excel application
excel.ScreenUpdating = True

# quit Excel application
excel.Quit()

# release COM object
del excel

envio_de_email(
    destinatarios=["marialaura.perez@security.cl",],
    concopia=[
        "dnunez@careassistance.com",
        "dgarcia@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx",
    activo=1,
) """

' import win32com.client as win32\nimport os\nimport shutil\n\nnombre_reporte = "Vida Security - Ferronor"\nnombre_carpeta = "F:/Reportes SF/Vida Security/"\nnombre_archivo_final = nombre_carpeta + str(dia) + "/" + nombre_reporte + ".xlsx"\nArchivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"\ncrear_carpeta(nombre_carpeta)\nactualizar(Archivo_origen)\nAsunto = "Detalle de Uso " + nombre_reporte + " - " + str(dia)\nshutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)\n\n# create Excel application object\nexcel = win32.gencache.EnsureDispatch(\'Excel.Application\')\n\n# set ScreenUpdating property to False to hide Excel application\nexcel.ScreenUpdating = False\n\n# open workbook\nworkbook = excel.Workbooks.Open(nombre_archivo_final)\n\n# do some work with the workbook\n\n# save and close workbook\nworkbook.Save()\nworkbook.Close()\n\n# set ScreenUpdating property to True to show Excel application\nexcel.ScreenUpdating = True\n\n# quit Excel applicati

In [43]:
""" import win32com.client as win32
import os
import shutil
import datetime as dt

def crear_carpeta(nombre_carpeta):
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)

def actualizar(Archivo_origen):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(Archivo_origen)
    wb.RefreshAll()
    wb.Save()
    wb.Close()

def envio_de_email(destinatarios, concopia, asunto, nombre_reporte, ruta_adjunto, nombre_adjunto):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = ";".join(destinatarios)
    mail.CC = ";".join(concopia)
    mail.Subject = asunto
    mail.Body = "Buen día,\n\nSe adjunta el detalle de uso de la aplicación " + nombre_reporte + " correspondiente al día de ayer.\n\nSaludos cordiales,\n\n"
    mail.Attachments.Add(ruta_adjunto, DisplayName=nombre_adjunto)
    mail.Send()

nombre_reporte = "Zurich - AMSA"
nombre_carpeta = "F:/Reportes SF/Zurich/"
nombre_archivo_final = nombre_carpeta + str(dt.date.today()) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dt.date.today())
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

envio_de_email(
    destinatarios=[
        "clara.angelica.munoz@zurich.com",
        "claudia.ivonne.zapata@zurich.com",
        "maria.moreno.poblete@zurich.com",
        "nadia.pino@zurich.com",
    ],
    concopia=[
        "dgarcia@careassistance.com",
        "dnunez@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx" """

' import win32com.client as win32\nimport os\nimport shutil\nimport datetime as dt\n\ndef crear_carpeta(nombre_carpeta):\n    if not os.path.exists(nombre_carpeta):\n        os.makedirs(nombre_carpeta)\n\ndef actualizar(Archivo_origen):\n    excel = win32.gencache.EnsureDispatch(\'Excel.Application\')\n    wb = excel.Workbooks.Open(Archivo_origen)\n    wb.RefreshAll()\n    wb.Save()\n    wb.Close()\n\ndef envio_de_email(destinatarios, concopia, asunto, nombre_reporte, ruta_adjunto, nombre_adjunto):\n    outlook = win32.Dispatch(\'outlook.application\')\n    mail = outlook.CreateItem(0)\n    mail.To = ";".join(destinatarios)\n    mail.CC = ";".join(concopia)\n    mail.Subject = asunto\n    mail.Body = "Buen día,\n\nSe adjunta el detalle de uso de la aplicación " + nombre_reporte + " correspondiente al día de ayer.\n\nSaludos cordiales,\n\n"\n    mail.Attachments.Add(ruta_adjunto, DisplayName=nombre_adjunto)\n    mail.Send()\n\nnombre_reporte = "Zurich - AMSA"\nnombre_carpeta = "F:/Repor

In [44]:
# campaña = 'MetLife'
query_empresacontactoconvenio = (
    "select Id,Paciente__r.IdentificationId__pc, Empresa__r.P_liza__c, Relaci_n_Con_Asegurado__c,N_Documento_Titular__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña
    + "'"
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)

NameError: name 'campaña' is not defined

In [ ]:
rut_yidEmCoCo = pd.DataFrame()
rut_yidEmCoCo["Id empreconcon"] = empresacontactoconvenio["Id"]
rut_yidEmCoCo["rut"] = empresacontactoconvenio["Paciente__r.IdentificationId__pc"]
rut_yidEmCoCo["Relacion"] = empresacontactoconvenio["Relaci_n_Con_Asegurado__c"]
rut_yidEmCoCo["rutT"] = empresacontactoconvenio["N_Documento_Titular__c"]
rut_yidEmCoCo["poliza"] = empresacontactoconvenio["Empresa__r.P_liza__c"]

rut_yidEmCoCo = rut_yidEmCoCo.drop_duplicates(
    ["rut", "Relacion", "rutT", "poliza"], keep="last"
)
rut_yidEmCoCo.head()

empresacontactoconvenio["RutEstaoNo"] = empresacontactoconvenio["Id"].isin(
    rut_yidEmCoCo["Id empreconcon"]
)
noestan = empresacontactoconvenio[(empresacontactoconvenio["RutEstaoNo"] == False)]

In [ ]:
noestan.shape
noestan.to_excel("C:/Users/jmartinez/Documents/noestandddd.xlsx", index=False)